In [2]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('C:\data\CardData_new.csv', encoding='cp949')
df.head()

,회원번호,남녀구분코드,연령,소지여부_신용,입회일자_신용,입회경과개월수_신용,탈회여부,회원여부_연체,수신거부여부_TM,수신거부여부_DM,...,캠페인접촉건수_R12M,캠페인접촉일수_R12M,캠페인접촉월수_R12M,접촉후경과월_R12M,보험콜대상월수_R12M,보험콜대상횟수_R12M,보험콜통화횟수_R12M,보험콜배정후경과월_R12M,보험콜통화후경과월_R12M,고객구분
0,YNtSN9Oa,2,64,1,20140709,54,1,0,0,0,...,3,3,2,5,5,5,9,0,0,21.Active_연속
1,Rijqdlzu,1,24,1,20170412,21,0,0,0,0,...,0,0,0,12,5,5,14,2,2,21.Active_연속
2,NOsf3TZS,2,43,1,20170426,21,1,0,1,0,...,0,0,0,12,0,0,0,12,12,21.Active_연속
3,7IPZGTZu,2,79,1,20141128,50,0,0,0,0,...,16,15,5,1,3,3,6,0,0,21.Active_연속
4,pEEsUF2u,2,82,1,20120905,76,0,0,0,0,...,4,4,3,1,1,1,4,3,2,21.Active_연속


In [4]:
df2 = pd.read_csv('C:/data/CardCategory.csv', encoding='euc-kr')
df2.head()

,_1순위카드ID,서비스업종코드,서비스코드명
0,1,209,온라인쇼핑할인
1,1,201,주유 할인
2,1,204,놀이공원할인
3,1,211,아파트할인
4,1,209,온라인쇼핑할인


In [5]:
df2 = df2[df2['서비스코드명'] != '_']
df2 = df2[df2['서비스코드명'] != '기타 포인트']
df2 = df2[df2['서비스코드명'] != '기타']


In [8]:
df2['서비스코드명'].unique()
card_ids = df2['_1순위카드ID'].unique()

In [5]:
df2['카드이용자수'] = df2['_1순위카드ID'].map(df['_1순위카드ID'].value_counts())

# 결과 확인
print(df2)

       _1순위카드ID 서비스업종코드   서비스코드명  카드이용자수
0             1     209  온라인쇼핑할인     334
1             1     201    주유 할인     334
2             1     204   놀이공원할인     334
3             1     211    아파트할인     334
4             1     209  온라인쇼핑할인     334
...         ...     ...      ...     ...
26680      1743     207     커피할인       4
26681      1743     901   기타 포인트       4
26682      1743     206   문화생활할인       4
26683      1743     901   기타 포인트       4
26684      1743     901   기타 포인트       4

[26685 rows x 4 columns]


In [9]:
def get_age_group(service_code):
    if service_code in ['놀이공원할인', '문화생활할인', '교육관련할인']:
        return '취미_여가'
    elif service_code in [ '주유할인', '자동차할인', '교통할인','대한항공 항공사 마일리지','아시아나 항공사 마일리지','통신할인', '아파트할인','기타']:
        return '교통/납부'
    elif service_code in ['전가맹점할인','커피할인','외식할인']:
        return 'daily'
    elif service_code in ['온라인쇼핑할인', '유통/마트 할인','가전할인']:
        return '구매'
    else:
        return '포인트/서비스'

# '6가지분류' 열 추가
df2['6가지분류'] = df2['서비스코드명'].apply(get_age_group)
age_group_counts = df2['6가지분류'].value_counts()
print(age_group_counts)

포인트/서비스    11802
취미_여가       7054
교통/납부       4912
daily       1940
구매           977
Name: 6가지분류, dtype: int64


In [36]:
for card_id in card_ids:
    print(f"Card ID {card_id}의 서비스코드명 빈도:")
    print(df2[df2['_1순위카드ID'] == card_id]['서비스코드명'].value_counts())
    print("\n")

Card ID 1의 서비스코드명 빈도:
주유 할인       10
놀이공원할인       6
온라인쇼핑할인      4
유통/마트 할인     3
아파트할인        2
교육관련할인       2
통신할인         2
문화생활할인       1
외식할인         1
기타 원금할인      1
Name: 서비스코드명, dtype: int64


Card ID 2의 서비스코드명 빈도:
주유 할인       10
놀이공원할인       6
온라인쇼핑할인      4
유통/마트 할인     4
커피할인         2
통신할인         2
문화생활할인       1
외식할인         1
기타 원금할인      1
교통할인         1
Name: 서비스코드명, dtype: int64


Card ID 3의 서비스코드명 빈도:
전가맹점할인    2
Name: 서비스코드명, dtype: int64


Card ID 4의 서비스코드명 빈도:
가전할인     2
외식할인     1
주유 할인    1
Name: 서비스코드명, dtype: int64


Card ID 7의 서비스코드명 빈도:
외식할인       2
문화생활할인     2
커피할인       2
기타         1
온라인쇼핑할인    1
기타 원금할인    1
Name: 서비스코드명, dtype: int64


Card ID 8의 서비스코드명 빈도:
통신할인       2
기타 원금할인    1
온라인쇼핑할인    1
문화생활할인     1
커피할인       1
외식할인       1
교통할인       1
Name: 서비스코드명, dtype: int64


Card ID 9의 서비스코드명 빈도:
문화생활할인     6
기타         1
온라인쇼핑할인    1
기타 포인트     1
Name: 서비스코드명, dtype: int64


Card ID 11의 서비스코드명 빈도:
문화생활할인      2
온라인쇼핑할인     2
유통/마트 할인    1
커피할인        

플래티늄서비스          271
놀이공원할인           147
문화생활할인           113
기타                70
대한항공 항공사 마일리지     35
주유 할인             20
교통할인              13
외식할인              11
자동차할인             10
기타 포인트             6
유통/마트 할인           3
Name: 서비스코드명, dtype: int64


Card ID 248의 서비스코드명 빈도:
자동차할인    4
Name: 서비스코드명, dtype: int64


Card ID 249의 서비스코드명 빈도:
놀이공원할인    4
자동차할인     4
Name: 서비스코드명, dtype: int64


Card ID 250의 서비스코드명 빈도:
외식할인        2
교육관련할인      1
가전할인        1
주유 할인       1
유통/마트 할인    1
Name: 서비스코드명, dtype: int64


Card ID 251의 서비스코드명 빈도:
놀이공원할인      14
기타          10
문화생활할인       6
유통/마트 할인     4
통신할인         2
기타 포인트       2
Name: 서비스코드명, dtype: int64


Card ID 252의 서비스코드명 빈도:
기타 포인트    6
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 253의 서비스코드명 빈도:
기타 포인트    7
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 254의 서비스코드명 빈도:
기타 포인트    5
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 255의 서비스코드명 빈도:
자동차할인       4
주유 할인       1
세이브포인트      1
기타 원금할인     1
유통/마트 할인    1
Name: 서비

자동차할인    4
기타       2
Name: 서비스코드명, dtype: int64


Card ID 520의 서비스코드명 빈도:
자동차할인    4
기타       2
Name: 서비스코드명, dtype: int64


Card ID 521의 서비스코드명 빈도:
놀이공원할인    5
자동차할인     4
주유 할인     3
문화생활할인    2
_         1
기타 포인트    1
Name: 서비스코드명, dtype: int64


Card ID 524의 서비스코드명 빈도:
놀이공원할인    15
기타 포인트     4
문화생활할인     1
Name: 서비스코드명, dtype: int64


Card ID 525의 서비스코드명 빈도:
기타 포인트    6
커피할인      2
Name: 서비스코드명, dtype: int64


Card ID 527의 서비스코드명 빈도:
기타 포인트    8
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 528의 서비스코드명 빈도:
기타 포인트    4
Name: 서비스코드명, dtype: int64


Card ID 529의 서비스코드명 빈도:
기타 포인트    3
기타        1
Name: 서비스코드명, dtype: int64


Card ID 530의 서비스코드명 빈도:
전가맹점할인    2
기타 포인트    1
기타        1
Name: 서비스코드명, dtype: int64


Card ID 531의 서비스코드명 빈도:
기타 포인트    8
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 533의 서비스코드명 빈도:
자동차할인    4
Name: 서비스코드명, dtype: int64


Card ID 534의 서비스코드명 빈도:
자동차할인     4
놀이공원할인    3
Name: 서비스코드명, dtype: int64


Card ID 535의 서비스코드명 빈도:
문화생활할인     3
커피할인       1
온라인쇼

온라인쇼핑할인    2
문화생활할인     1
커피할인       1
외식할인       1
Name: 서비스코드명, dtype: int64


Card ID 773의 서비스코드명 빈도:
기타 포인트     9
기타 원금할인    3
커피할인       1
주유 할인      1
Name: 서비스코드명, dtype: int64


Card ID 774의 서비스코드명 빈도:
기타 포인트    1
Name: 서비스코드명, dtype: int64


Card ID 775의 서비스코드명 빈도:
자동차할인     4
놀이공원할인    1
Name: 서비스코드명, dtype: int64


Card ID 776의 서비스코드명 빈도:
문화생활할인    6
자동차할인     4
기타 포인트    1
Name: 서비스코드명, dtype: int64


Card ID 777의 서비스코드명 빈도:
문화생활할인    6
자동차할인     4
기타 포인트    1
Name: 서비스코드명, dtype: int64


Card ID 778의 서비스코드명 빈도:
자동차할인            4
대한항공 항공사 마일리지    1
Name: 서비스코드명, dtype: int64


Card ID 780의 서비스코드명 빈도:
기타 포인트     7
커피할인       1
기타 원금할인    1
Name: 서비스코드명, dtype: int64


Card ID 781의 서비스코드명 빈도:
기타 포인트    6
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 782의 서비스코드명 빈도:
기타 포인트    7
외식할인      1
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 783의 서비스코드명 빈도:
기타 포인트    7
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 784의 서비스코드명 빈도:
기타 포인트    9
커피할인      1
Name: 서비스코드명, dtype:

교통할인        2
유통/마트 할인    1
Name: 서비스코드명, dtype: int64


Card ID 1032의 서비스코드명 빈도:
교통할인        2
유통/마트 할인    1
Name: 서비스코드명, dtype: int64


Card ID 1033의 서비스코드명 빈도:
교통할인        2
유통/마트 할인    1
Name: 서비스코드명, dtype: int64


Card ID 1034의 서비스코드명 빈도:
교통할인        2
유통/마트 할인    1
Name: 서비스코드명, dtype: int64


Card ID 1035의 서비스코드명 빈도:
전가맹점할인    2
기타        1
기타 포인트    1
Name: 서비스코드명, dtype: int64


Card ID 1036의 서비스코드명 빈도:
기타 포인트    6
커피할인      1
주유 할인     1
Name: 서비스코드명, dtype: int64


Card ID 1037의 서비스코드명 빈도:
기타 포인트    6
커피할인      1
가전할인      1
Name: 서비스코드명, dtype: int64


Card ID 1038의 서비스코드명 빈도:
기타 포인트    8
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1039의 서비스코드명 빈도:
기타 포인트    7
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1041의 서비스코드명 빈도:
자동차할인      4
플래티늄서비스    3
세이브포인트     2
기타 포인트     1
Name: 서비스코드명, dtype: int64


Card ID 1045의 서비스코드명 빈도:
기타 포인트    6
커피할인      2
Name: 서비스코드명, dtype: int64


Card ID 1046의 서비스코드명 빈도:
기타 포인트    8
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1

자동차할인       4
외식할인        4
놀이공원할인      3
유통/마트 할인    2
커피할인        2
주유 할인       2
기타          1
문화생활할인      1
통신할인        1
온라인쇼핑할인     1
교통할인        1
Name: 서비스코드명, dtype: int64


Card ID 1298의 서비스코드명 빈도:
기타 포인트    6
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1299의 서비스코드명 빈도:
기타 포인트    6
주유 할인     1
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1300의 서비스코드명 빈도:
기타 포인트    7
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1301의 서비스코드명 빈도:
기타 포인트    6
커피할인      2
Name: 서비스코드명, dtype: int64


Card ID 1302의 서비스코드명 빈도:
기타 포인트    9
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1303의 서비스코드명 빈도:
기타 포인트    7
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1304의 서비스코드명 빈도:
문화생활할인     3
커피할인       1
온라인쇼핑할인    1
Name: 서비스코드명, dtype: int64


Card ID 1309의 서비스코드명 빈도:
놀이공원할인    8
자동차할인     4
기타 포인트    1
Name: 서비스코드명, dtype: int64


Card ID 1310의 서비스코드명 빈도:
기타 포인트    8
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1312의 서비스코드명 빈도:
자동차할인     4
놀이공원할인    2
Name: 서비스코드명, dtype: int64


기타 포인트    10
커피할인       1
Name: 서비스코드명, dtype: int64


Card ID 1557의 서비스코드명 빈도:
기타 포인트    7
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1558의 서비스코드명 빈도:
기타 포인트    28
커피할인       3
통신할인       2
Name: 서비스코드명, dtype: int64


Card ID 1559의 서비스코드명 빈도:
기타 포인트    9
통신할인      1
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1560의 서비스코드명 빈도:
기타 포인트    7
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1561의 서비스코드명 빈도:
기타 포인트    9
커피할인      1
Name: 서비스코드명, dtype: int64


Card ID 1562의 서비스코드명 빈도:
기타 포인트     7
커피할인       1
기타 원금할인    1
Name: 서비스코드명, dtype: int64


Card ID 1563의 서비스코드명 빈도:
기타 포인트     8
기타 원금할인    1
커피할인       1
Name: 서비스코드명, dtype: int64


Card ID 1564의 서비스코드명 빈도:
기타 포인트    15
커피할인       3
Name: 서비스코드명, dtype: int64


Card ID 1565의 서비스코드명 빈도:
기타 포인트    4
Name: 서비스코드명, dtype: int64


Card ID 1567의 서비스코드명 빈도:
기타 포인트    1
Name: 서비스코드명, dtype: int64


Card ID 1568의 서비스코드명 빈도:
기타 포인트    13
Name: 서비스코드명, dtype: int64


Card ID 1569의 서비스코드명 빈도:
기타 포인트    13
Name: 서비스코드명, dtype: int6

In [38]:


# 각 카드별로 빈도 계산하고 가장 높은 빈도의 서비스코드명을 새로운 칼럼으로 추가
card_ids = df2['_1순위카드ID'].unique()
df3 = pd.DataFrame(columns=['Card ID', '서비스코드명 빈도'])

for card_id in card_ids:
    card_data = df2[df2['_1순위카드ID'] == card_id]['서비스코드명']
    max_service_code = card_data.value_counts().idxmax()
    max_service_code_count = card_data.value_counts().max()
    df_temp = pd.DataFrame({'Card ID': [card_id], '서비스코드명 빈도': [f"{max_service_code}({max_service_code_count})"]})
    df3 = pd.concat([df3, df_temp], ignore_index=True)

print(df3)

     Card ID  서비스코드명 빈도
0          1  주유 할인(10)
1          2  주유 할인(10)
2          3  전가맹점할인(2)
3          4    가전할인(2)
4          7    외식할인(2)
...      ...        ...
1458    1739  놀이공원할인(5)
1459    1740   자동차할인(4)
1460    1741    가전할인(3)
1461    1742   자동차할인(4)
1462    1743  기타 포인트(5)

[1463 rows x 2 columns]


In [201]:
df3 = pd.DataFrame(columns=['Card ID', 'toppoint'])

for card_id in card_ids:
    card_data = df2[df2['_1순위카드ID'] == card_id]['서비스코드명'].value_counts()
    
    if not card_data.empty:  # 비어있지 않은 경우에만 처리
        toppoint_value = card_data.idxmax()
        df_temp = pd.DataFrame({'Card ID': [card_id], 'toppoint': [toppoint_value]})
        df3 = pd.concat([df3, df_temp], ignore_index=True)

# 나머지 코드는 동일
df3['카드이용자수'] = df3['Card ID'].map(df['_1순위카드ID'].value_counts())
df3.groupby('toppoint')['카드이용자수'].sum()

toppoint
가전할인             15207
교육관련할인             403
교통할인               773
기타 원금할인          11406
놀이공원할인           43710
대한항공 항공사 마일리지       26
문화생활할인            8094
아파트할인                5
온라인쇼핑할인          15219
외식할인              4102
유통/마트 할인           460
자동차할인             8527
전가맹점할인            3156
주유 할인             8081
캐쉬백서비스            6967
커피할인             46184
통신할인             29696
플래티늄서비스          54911
Name: 카드이용자수, dtype: int64

In [202]:
df3 = pd.DataFrame(columns=['Card ID', 'toppoint'])

for card_id in card_ids:
    card_data = df2[df2['_1순위카드ID'] == card_id]['서비스코드명'].value_counts()
    
    if not card_data.empty:  # 비어있지 않은 경우에만 처리
        toppoint_value = card_data.idxmax()
        df_temp = pd.DataFrame({'Card ID': [card_id], 'toppoint': [toppoint_value]})
        df3 = pd.concat([df3, df_temp], ignore_index=True)

# 나머지 코드는 동일
df3['카드이용자수'] = df3['Card ID'].map(df['_1순위카드ID'].value_counts())
df3.groupby('toppoint')['카드이용자수'].max()

toppoint
가전할인              3139
교육관련할인             334
교통할인               608
기타 원금할인           8055
놀이공원할인           18737
대한항공 항공사 마일리지       25
문화생활할인            5978
아파트할인                5
온라인쇼핑할인           9097
외식할인              2956
유통/마트 할인           329
자동차할인             1838
전가맹점할인            1418
주유 할인             5645
캐쉬백서비스            1197
커피할인              9769
통신할인             11856
플래티늄서비스          17092
Name: 카드이용자수, dtype: int64

In [179]:
for card_id in card_ids:
    print(f"Card ID {card_id}의 서비스코드명 빈도:")
    print(df2[df2['_1순위카드ID'] == card_id]['6가지분류'].value_counts())
    print("\n")

Card ID 1의 서비스코드명 빈도:
포인트/서비스    11
취미_여가       9
구매          7
교통/납부       4
daily       1
Name: 6가지분류, dtype: int64


Card ID 2의 서비스코드명 빈도:
포인트/서비스    11
구매          8
취미_여가       7
daily       3
교통/납부       3
Name: 6가지분류, dtype: int64


Card ID 3의 서비스코드명 빈도:
daily    2
Name: 6가지분류, dtype: int64


Card ID 4의 서비스코드명 빈도:
구매         2
daily      1
포인트/서비스    1
Name: 6가지분류, dtype: int64


Card ID 7의 서비스코드명 빈도:
daily      4
취미_여가      2
교통/납부      1
구매         1
포인트/서비스    1
Name: 6가지분류, dtype: int64


Card ID 8의 서비스코드명 빈도:
교통/납부      3
daily      2
포인트/서비스    1
구매         1
취미_여가      1
Name: 6가지분류, dtype: int64


Card ID 9의 서비스코드명 빈도:
취미_여가    6
교통/납부    1
구매       1
Name: 6가지분류, dtype: int64


Card ID 11의 서비스코드명 빈도:
구매       3
취미_여가    2
daily    1
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 12의 서비스코드명 빈도:
구매       3
취미_여가    2
교통/납부    1
daily    1
Name: 6가지분류, dtype: int64


Card ID 13의 서비스코드명 빈도:
구매       3
취미_여가    2
교통/납부    1
daily    1
Name: 6가지분류, dtype: int64


Card ID 14의 

취미_여가      128
포인트/서비스    122
교통/납부       34
daily       10
구매           2
Name: 6가지분류, dtype: int64


Card ID 262의 서비스코드명 빈도:
교통/납부      4
포인트/서비스    2
취미_여가      1
Name: 6가지분류, dtype: int64


Card ID 263의 서비스코드명 빈도:
포인트/서비스    1
Name: 6가지분류, dtype: int64


Card ID 264의 서비스코드명 빈도:
포인트/서비스    1
Name: 6가지분류, dtype: int64


Card ID 265의 서비스코드명 빈도:
취미_여가    3
구매       1
daily    1
Name: 6가지분류, dtype: int64


Card ID 266의 서비스코드명 빈도:
교통/납부    6
daily    3
취미_여가    3
Name: 6가지분류, dtype: int64


Card ID 267의 서비스코드명 빈도:
취미_여가      76
포인트/서비스    39
교통/납부      23
daily       1
구매          1
Name: 6가지분류, dtype: int64


Card ID 268의 서비스코드명 빈도:
취미_여가      56
포인트/서비스    36
교통/납부      14
daily       1
구매          1
Name: 6가지분류, dtype: int64


Card ID 269의 서비스코드명 빈도:
취미_여가      66
포인트/서비스    57
교통/납부      16
구매          1
daily       1
Name: 6가지분류, dtype: int64


Card ID 270의 서비스코드명 빈도:
취미_여가      64
포인트/서비스    53
교통/납부      15
구매          1
daily       1
Name: 6가지분류, dtype: int64


Card ID 272의 서비스코드

취미_여가      67
포인트/서비스    59
교통/납부      17
daily       5
구매          1
Name: 6가지분류, dtype: int64


Card ID 557의 서비스코드명 빈도:
취미_여가      24
교통/납부      12
포인트/서비스     9
daily       4
Name: 6가지분류, dtype: int64


Card ID 558의 서비스코드명 빈도:
교통/납부    4
Name: 6가지분류, dtype: int64


Card ID 559의 서비스코드명 빈도:
취미_여가    2
Name: 6가지분류, dtype: int64


Card ID 561의 서비스코드명 빈도:
교통/납부    6
Name: 6가지분류, dtype: int64


Card ID 562의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 563의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 564의 서비스코드명 빈도:
교통/납부      9
취미_여가      6
포인트/서비스    3
Name: 6가지분류, dtype: int64


Card ID 565의 서비스코드명 빈도:
취미_여가      6
교통/납부      6
포인트/서비스    3
Name: 6가지분류, dtype: int64


Card ID 566의 서비스코드명 빈도:
교통/납부      6
취미_여가      6
포인트/서비스    3
Name: 6가지분류, dtype: int64


Card ID 567의 서비스코드명 빈도:
Series([], Name: 6가지분류, dtype: int64)


Card ID 568의 서비스코드명 빈도:
daily    1
취미_여가    1
Name: 6가지분류, dtype: int64


Card ID 569의 서비스코드명 빈도:
취미_여가    1
daily    1
Name: 6가지분류, dtype: in

취미_여가    3
daily    1
구매       1
Name: 6가지분류, dtype: int64


Card ID 826의 서비스코드명 빈도:
취미_여가    3
구매       1
daily    1
Name: 6가지분류, dtype: int64


Card ID 827의 서비스코드명 빈도:
교통/납부    11
취미_여가     4
Name: 6가지분류, dtype: int64


Card ID 828의 서비스코드명 빈도:
교통/납부      7
취미_여가      7
포인트/서비스    3
Name: 6가지분류, dtype: int64


Card ID 829의 서비스코드명 빈도:
교통/납부    1
daily    1
Name: 6가지분류, dtype: int64


Card ID 830의 서비스코드명 빈도:
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 831의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 832의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 833의 서비스코드명 빈도:
daily      1
포인트/서비스    1
Name: 6가지분류, dtype: int64


Card ID 834의 서비스코드명 빈도:
daily    3
구매       2
취미_여가    2
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 835의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 836의 서비스코드명 빈도:
포인트/서비스    5
daily      1
Name: 6가지분류, dtype: int64


Card ID 837의 서비스코드명 빈도:
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 838의 서비스코드명 빈도:
구매         4
포인트/서비스  

포인트/서비스    23
교통/납부       4
취미_여가       2
Name: 6가지분류, dtype: int64


Card ID 1088의 서비스코드명 빈도:
포인트/서비스    30
교통/납부       4
취미_여가       2
Name: 6가지분류, dtype: int64


Card ID 1089의 서비스코드명 빈도:
취미_여가      11
교통/납부       5
포인트/서비스     1
daily       1
Name: 6가지분류, dtype: int64


Card ID 1090의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1091의 서비스코드명 빈도:
daily    1
취미_여가    1
Name: 6가지분류, dtype: int64


Card ID 1092의 서비스코드명 빈도:
취미_여가    5
교통/납부    4
Name: 6가지분류, dtype: int64


Card ID 1093의 서비스코드명 빈도:
교통/납부      5
daily      2
포인트/서비스    1
Name: 6가지분류, dtype: int64


Card ID 1094의 서비스코드명 빈도:
취미_여가    5
daily    3
구매       1
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1095의 서비스코드명 빈도:
포인트/서비스    46
취미_여가      10
daily       6
교통/납부       2
구매          2
Name: 6가지분류, dtype: int64


Card ID 1096의 서비스코드명 빈도:
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1097의 서비스코드명 빈도:
취미_여가      9
교통/납부      6
daily      3
포인트/서비스    1
Name: 6가지분류, dtype: int64


Card ID 1098의 서비스코드명 빈도:
daily 

포인트/서비스    4
Name: 6가지분류, dtype: int64


Card ID 1371의 서비스코드명 빈도:
교통/납부    4
취미_여가    2
Name: 6가지분류, dtype: int64


Card ID 1373의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1374의 서비스코드명 빈도:
취미_여가    6
교통/납부    4
Name: 6가지분류, dtype: int64


Card ID 1375의 서비스코드명 빈도:
취미_여가    6
교통/납부    4
Name: 6가지분류, dtype: int64


Card ID 1378의 서비스코드명 빈도:
교통/납부    4
Name: 6가지분류, dtype: int64


Card ID 1379의 서비스코드명 빈도:
교통/납부    4
취미_여가    3
Name: 6가지분류, dtype: int64


Card ID 1380의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1381의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1382의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1383의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1384의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1385의 서비스코드명 빈도:
취미_여가      6
구매         4
포인트/서비스    2
daily      2
교통/납부      1
Name: 6가지분류, dtype: int64


Card ID 1386의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1387의 서비스코드명 빈도:
dail

daily    1
Name: 6가지분류, dtype: int64


Card ID 1638의 서비스코드명 빈도:
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1639의 서비스코드명 빈도:
daily    2
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1640의 서비스코드명 빈도:
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1641의 서비스코드명 빈도:
daily    2
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1643의 서비스코드명 빈도:
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1644의 서비스코드명 빈도:
Series([], Name: 6가지분류, dtype: int64)


Card ID 1645의 서비스코드명 빈도:
daily    2
Name: 6가지분류, dtype: int64


Card ID 1646의 서비스코드명 빈도:
daily    1
Name: 6가지분류, dtype: int64


Card ID 1650의 서비스코드명 빈도:
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1651의 서비스코드명 빈도:
daily    2
교통/납부    1
Name: 6가지분류, dtype: int64


Card ID 1653의 서비스코드명 빈도:
Series([], Name: 6가지분류, dtype: int64)


Card ID 1654의 서비스코드명 빈도:
교통/납부    2
Name: 6가지분류, dtype: int64


Card ID 1655의 서비스코드명 빈도:
daily    4
교통/납부    2
Name: 6가지분류, dtype: int64


Card ID 1656의 서비스코드명 빈도:
취미_여가    3
daily    1
구매       1
Name: 6가지분류, dtype: int64

In [34]:
df2

,_1순위카드ID,서비스업종코드,서비스코드명,카드이용자수,6가지분류
0,1,209,온라인쇼핑할인,334,구매
1,1,201,주유 할인,334,포인트/서비스
2,1,204,놀이공원할인,334,취미_여가
3,1,211,아파트할인,334,교통/납부
4,1,209,온라인쇼핑할인,334,구매
...,...,...,...,...,...
26680,1743,207,커피할인,4,daily
26681,1743,901,기타 포인트,4,포인트/서비스
26682,1743,206,문화생활할인,4,취미_여가
26683,1743,901,기타 포인트,4,포인트/서비스


In [7]:
df3 = pd.DataFrame(columns=['Card ID', 'toppoint'])

for card_id in card_ids:
    card_data = df2[df2['_1순위카드ID'] == card_id]['6가지분류'].value_counts()
    
    if not card_data.empty:  # 비어있지 않은 경우에만 처리
        toppoint_value = card_data.idxmax()
        df_temp = pd.DataFrame({'Card ID': [card_id], 'toppoint': [toppoint_value]})
        df3 = pd.concat([df3, df_temp], ignore_index=True)

# 나머지 코드는 동일
df3['카드이용자수'] = df3['Card ID'].map(df['_1순위카드ID'].value_counts())
df3.groupby('toppoint')['카드이용자수'].sum()

NameError: name 'card_ids' is not defined

In [181]:
df = df.merge(df3[['Card ID', 'toppoint']], how='left', left_on='_1순위카드ID', right_on='Card ID')

# 결과 확인
print(df)

            회원번호  남녀구분코드  연령  소지여부_신용   입회일자_신용  입회경과개월수_신용  탈회여부  회원여부_연체  \
0       YNtSN9Oa       2  64        1  20140709          54     1        0   
1       Rijqdlzu       1  24        1  20170412          21     0        0   
2       NOsf3TZS       2  43        1  20170426          21     1        0   
3       7IPZGTZu       2  79        1  20141128          50     0        0   
4       pEEsUF2u       2  82        1  20120905          76     0        0   
...          ...     ...  ..      ...       ...         ...   ...      ...   
299995  Gso9Jlid       2  47        1  20170630          18     0        0   
299996  EDQYlTTX       2  69        1  20150417          45     0        0   
299997  82GwFS1D       1  36        1  20120608          79     0        0   
299998  LUYeNjki       1  48        1  20120508          80     1        0   
299999  bnKXePYO       2  36        1  20180222          11     1        0   

        수신거부여부_TM  수신거부여부_DM  ...  캠페인접촉월수_R12M  접촉후경과월_R12M 보험

In [182]:
df['toppoint'].fillna('구매', inplace=True)

# 결과 확인
print(df)



            회원번호  남녀구분코드  연령  소지여부_신용   입회일자_신용  입회경과개월수_신용  탈회여부  회원여부_연체  \
0       YNtSN9Oa       2  64        1  20140709          54     1        0   
1       Rijqdlzu       1  24        1  20170412          21     0        0   
2       NOsf3TZS       2  43        1  20170426          21     1        0   
3       7IPZGTZu       2  79        1  20141128          50     0        0   
4       pEEsUF2u       2  82        1  20120905          76     0        0   
...          ...     ...  ..      ...       ...         ...   ...      ...   
299995  Gso9Jlid       2  47        1  20170630          18     0        0   
299996  EDQYlTTX       2  69        1  20150417          45     0        0   
299997  82GwFS1D       1  36        1  20120608          79     0        0   
299998  LUYeNjki       1  48        1  20120508          80     1        0   
299999  bnKXePYO       2  36        1  20180222          11     1        0   

        수신거부여부_TM  수신거부여부_DM  ...  캠페인접촉월수_R12M  접촉후경과월_R12M 보험

In [183]:
df['toppoint'].value_counts()

포인트/서비스    71749
구매         70079
교통/납부      57017
daily      55912
취미_여가      45243
Name: toppoint, dtype: int64

In [184]:
import os

# 데이터프레임을 CSV 파일로 저장
output_folder = r'C:\Users\shinseungho\Desktop\프로젝트2'  # 폴더 경로 설정
output_file = os.path.join(output_folder, 'new_dataframe.csv')

df.to_csv(output_file, index=False, encoding='cp949')

In [14]:
# 빈 DataFrame 생성
df3 = pd.DataFrame(columns=df2.columns)

# 각 card_id에 대해 반복
for card_id in card_ids:
    card_df = df2[df2['_1순위카드ID'] == card_id]
    card_counts = card_df.value_counts()
    
    # 시리즈를 데이터프레임으로 변환하여 첫 번째 행 추가
    first_row = card_counts.index[0]
    df3 = df3.append(pd.DataFrame([first_row], columns=df3.columns), ignore_index=True)

# df3 출력
print(df3)

     _1순위카드ID 서비스업종코드  서비스코드명 카드이용자수    6가지분류
0           1     201   주유 할인    334  포인트/서비스
1           2     201   주유 할인     53  포인트/서비스
2           3     214  전가맹점할인     20    daily
3           4     212    가전할인      2       구매
4           7     205    외식할인      6    daily
...       ...     ...     ...    ...      ...
2921     1739     204  놀이공원할인   5143    취미_여가
2922     1740     204  놀이공원할인    199    취미_여가
2923     1741     212    가전할인     25       구매
2924     1742     210   자동차할인     13    교통/납부
2925     1743     901  기타 포인트      4  포인트/서비스

[2926 rows x 5 columns]


In [15]:
# 빈 DataFrame 생성
df4 = pd.DataFrame(columns=['Card ID', 'top_service_code', 'top_service_code_count'])

# 각 card_id에 대해 반복
for card_id in card_ids:
    card_data = df2[df2['_1순위카드ID'] == card_id]['서비스코드명'].value_counts()
    
    # 가장 많이 나타나는 값(top_service_code)과 해당 카드의 ID를 저장
    top_service_code = card_data.idxmax()
    top_service_code_count = card_data.max()
    
    df_temp = pd.DataFrame({'Card ID': [card_id], 'top_service_code': [top_service_code], 'top_service_code_count': [top_service_code_count]})
    df4 = pd.concat([df4, df_temp], ignore_index=True)

# 결과 확인
print(df4)

     Card ID top_service_code top_service_code_count
0          1            주유 할인                     10
1          2            주유 할인                     10
2          3           전가맹점할인                      2
3          4             가전할인                      2
4          7             외식할인                      2
...      ...              ...                    ...
1458    1739           놀이공원할인                      5
1459    1740            자동차할인                      4
1460    1741             가전할인                      3
1461    1742            자동차할인                      4
1462    1743           기타 포인트                      5

[1463 rows x 3 columns]


In [19]:
selected_data = df4[df4['top_service_code'] == '주유 할인']
print(selected_data)

    Card ID top_service_code top_service_code_count
0         1            주유 할인                     10
1         2            주유 할인                     10
900    1086            주유 할인                      7
958    1152            주유 할인                      6


In [ ]:
selected_data = df4[df4['top_service_code'] == '주유 할인']
print(selected_data)

In [28]:
# 빈 DataFrame 생성
max_count_df = pd.DataFrame(columns=['Card ID', 'top_service_code', 'top_service_code_count'])

# 각 card_id에 대해 반복
for card_id in card_ids:
    card_data = df2[df2['_1순위카드ID'] == card_id]['서비스코드명'].value_counts()
    
    # 가장 많이 나타나는 서비스 코드명(top_service_code)과 해당 카드의 ID를 저장
    top_service_code = card_data.idxmax()
    top_service_code_count = card_data.max()
    
    df_temp = pd.DataFrame({'Card ID': [card_id], 'top_service_code': [top_service_code], 'top_service_code_count': [top_service_code_count]})
    max_count_df = pd.concat([max_count_df, df_temp], ignore_index=True)

# 최댓값을 가진 데이터만 선택
unique_service_codes = max_count_df['top_service_code'].unique()
max_count_rows = pd.DataFrame(columns=['Card ID', 'top_service_code', 'top_service_code_count'])
for service_code in unique_service_codes:
    filtered_rows = max_count_df[max_count_df['top_service_code'] == service_code]
    max_count = filtered_rows['top_service_code_count'].max()
    max_count_rows = pd.concat([max_count_rows, filtered_rows[filtered_rows['top_service_code_count'] == max_count]])

print(max_count_rows)

     Card ID top_service_code top_service_code_count
0          1            주유 할인                     10
1          2            주유 할인                     10
100      130           전가맹점할인                      8
184      230             가전할인                      4
1165    1400             가전할인                      4
239      296             외식할인                      9
259      328             외식할인                      9
180      226             통신할인                     16
243      300             통신할인                     16
321      411           문화생활할인                     15
934     1126           문화생활할인                     15
94       123          온라인쇼핑할인                     54
275      352           기타 포인트                    121
197      247          플래티늄서비스                    271
106      136           놀이공원할인                    104
648      791           캐쉬백서비스                     29
1219    1461            자동차할인                     28
1220    1462            자동차할인                 

In [42]:
df2[df2['서비스코드명']=='아파트할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명
1152      211      아파트할인     4
1         211      아파트할인     2
22        211      아파트할인     2
112       211      아파트할인     2
113       211      아파트할인     2
114       211      아파트할인     2
115       211      아파트할인     2
116       211      아파트할인     2
928       211      아파트할인     2
929       211      아파트할인     2
306       211      아파트할인     1
374       211      아파트할인     1
397       211      아파트할인     1
855       211      아파트할인     1
919       211      아파트할인     1
1394      211      아파트할인     1
dtype: int64

In [7]:
df2[df2['서비스코드명']=='주유 할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명
136       201      주유 할인     26
260       201      주유 할인     20
247       201      주유 할인     20
25        201      주유 할인     19
1007      201      주유 할인     18
                             ..
646       201      주유 할인      1
647       201      주유 할인      1
651       201      주유 할인      1
652       201      주유 할인      1
1741      201      주유 할인      1
Length: 332, dtype: int64

In [8]:
len(df[df['교통_주유이용금액']>0])

91400

In [12]:
len(df[(df['_1순위카드ID'] == 260) & (df['교통_주유이용금액'] > 0)])

247

In [13]:
len(df[(df['_1순위카드ID'] == 136) & (df['교통_주유이용금액'] > 0)])

5294

In [14]:
len(df[df['교통_버스지하철이용금액']>0])

83764

In [21]:
len(df[(df['_1순위카드ID'] ==226) & (df['납부_통신비이용금액'] > 0)])

2914

In [20]:
len(df[df['_1순위카드ID'] == 226])

3058

In [23]:
df2[df2['_1순위카드ID']==226].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명 
226       202      통신할인       16
          201      주유 할인       2
          205      외식할인        2
          206      문화생활할인      1
          207      커피할인        1
          900      기타 원금할인     1
dtype: int64

In [22]:
df2[df2['서비스코드명']=='통신할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명
136       202      통신할인      17
226       202      통신할인      16
300       202      통신할인      16
217       202      통신할인      14
229       202      통신할인      12
                             ..
201       202      통신할인       1
844       202      통신할인       1
202       202      통신할인       1
793       202      통신할인       1
474       202      통신할인       1
Length: 187, dtype: int64

In [24]:
df2[df2['_1순위카드ID']==226].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명 
226       202      통신할인       16
          201      주유 할인       2
          205      외식할인        2
          206      문화생활할인      1
          207      커피할인        1
          900      기타 원금할인     1
dtype: int64

In [25]:
len(df[df['_1순위카드ID'] == 226])

3058

In [26]:
len(df[(df['_1순위카드ID'] ==226) & (df['납부_통신비이용금액'] > 0)])

2914

In [49]:
df2[df2['서비스코드명']=='교통할인'].value_counts() 

_1순위카드ID  서비스업종코드  서비스코드명
136       208      교통할인      17
1061      208      교통할인      13
247       208      교통할인      13
25        208      교통할인      11
1008      208      교통할인       8
                             ..
695       208      교통할인       1
707       208      교통할인       1
716       208      교통할인       1
754       208      교통할인       1
874       208      교통할인       1
Length: 195, dtype: int64

In [50]:
df2[df2['서비스코드명']=='문화생활할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명
247       206      문화생활할인    113
1061      206      문화생활할인    103
1007      206      문화생활할인     80
1008      206      문화생활할인     76
260       206      문화생활할인     70
                            ... 
656       206      문화생활할인      1
653       206      문화생활할인      1
652       206      문화생활할인      1
651       206      문화생활할인      1
1743      206      문화생활할인      1
Length: 479, dtype: int64

In [51]:
df2[df2['서비스코드명']=='유통/마트 할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명  
123       203      유통/마트 할인    18
115       203      유통/마트 할인     9
116       203      유통/마트 할인     9
114       203      유통/마트 할인     7
484       203      유통/마트 할인     6
                               ..
205       203      유통/마트 할인     1
206       203      유통/마트 할인     1
207       203      유통/마트 할인     1
925       203      유통/마트 할인     1
1031      203      유통/마트 할인     1
Length: 169, dtype: int64

In [53]:
df2[df2['서비스코드명']=='온라인쇼핑할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명 
123       209      온라인쇼핑할인    54
114       209      온라인쇼핑할인    36
1708      209      온라인쇼핑할인    18
121       209      온라인쇼핑할인    18
115       209      온라인쇼핑할인    18
                              ..
718       209      온라인쇼핑할인     1
547       209      온라인쇼핑할인     1
517       209      온라인쇼핑할인     1
486       209      온라인쇼핑할인     1
535       209      온라인쇼핑할인     1
Length: 123, dtype: int64

In [54]:
df2[df2['서비스코드명']=='전가맹점할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명
130       214      전가맹점할인    8
605       214      전가맹점할인    6
1065      214      전가맹점할인    6
217       214      전가맹점할인    5
1655      214      전가맹점할인    4
                            ..
1269      214      전가맹점할인    1
1274      214      전가맹점할인    1
1337      214      전가맹점할인    1
755       214      전가맹점할인    1
1717      214      전가맹점할인    1
Length: 78, dtype: int64

In [56]:
df2[df2['서비스코드명']=='아시아나 항공사 마일리지'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명       
1462      102      아시아나 항공사 마일리지    7
1061      102      아시아나 항공사 마일리지    6
1014      102      아시아나 항공사 마일리지    5
1059      102      아시아나 항공사 마일리지    3
1060      102      아시아나 항공사 마일리지    2
283       102      아시아나 항공사 마일리지    1
587       102      아시아나 항공사 마일리지    1
695       102      아시아나 항공사 마일리지    1
1052      102      아시아나 항공사 마일리지    1
1108      102      아시아나 항공사 마일리지    1
1244      102      아시아나 항공사 마일리지    1
1286      102      아시아나 항공사 마일리지    1
dtype: int64

In [57]:
df2[df2['서비스코드명']=='대한항공 항공사 마일리지'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명       
247       101      대한항공 항공사 마일리지    35
246       101      대한항공 항공사 마일리지    10
1008      101      대한항공 항공사 마일리지     9
1007      101      대한항공 항공사 마일리지     8
1461      101      대한항공 항공사 마일리지     7
1016      101      대한항공 항공사 마일리지     7
765       101      대한항공 항공사 마일리지     6
176       101      대한항공 항공사 마일리지     6
1003      101      대한항공 항공사 마일리지     6
180       101      대한항공 항공사 마일리지     5
1005      101      대한항공 항공사 마일리지     3
159       101      대한항공 항공사 마일리지     2
1460      101      대한항공 항공사 마일리지     2
1051      101      대한항공 항공사 마일리지     1
60        101      대한항공 항공사 마일리지     1
1107      101      대한항공 항공사 마일리지     1
1113      101      대한항공 항공사 마일리지     1
1243      101      대한항공 항공사 마일리지     1
1006      101      대한항공 항공사 마일리지     1
1432      101      대한항공 항공사 마일리지     1
1344      101      대한항공 항공사 마일리지     1
885       101      대한항공 항공사 마일리지     1
1004      101      대한항공 항공사 마일리지     1
66        101      대한항공 항공사 마일리지     1
884       101      대한항공 항공사 마일리

In [58]:
df2[df2['서비스코드명']=='가전할인'].value_counts()

_1순위카드ID  서비스업종코드  서비스코드명
230       212      가전할인      4
1400      212      가전할인      4
1741      212      가전할인      3
1405      212      가전할인      3
240       212      가전할인      3
299       212      가전할인      3
1326      212      가전할인      3
838       212      가전할인      3
101       212      가전할인      2
1731      212      가전할인      2
1722      212      가전할인      2
1721      212      가전할인      2
1720      212      가전할인      2
1401      212      가전할인      2
1328      212      가전할인      2
4         212      가전할인      2
135       212      가전할인      2
144       212      가전할인      2
191       212      가전할인      2
970       212      가전할인      1
1403      212      가전할인      1
143       212      가전할인      1
227       212      가전할인      1
1404      212      가전할인      1
250       212      가전할인      1
1197      212      가전할인      1
317       212      가전할인      1
442       212      가전할인      1
1327      212      가전할인      1
768       212      가전할인      1
1325      212      가전할인      1
1199      212

In [59]:
df2[df2['서비스코드명']=='기타 원금할인'].value_counts() 

_1순위카드ID  서비스업종코드  서비스코드명 
114       900      기타 원금할인    6
336       900      기타 원금할인    4
836       900      기타 원금할인    4
1400      900      기타 원금할인    4
1511      900      기타 원금할인    4
                             ..
539       900      기타 원금할인    1
493       900      기타 원금할인    1
486       900      기타 원금할인    1
485       900      기타 원금할인    1
1731      900      기타 원금할인    1
Length: 131, dtype: int64